In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!cp gdrive/MyDrive/dontpatronizeme_pcl.tsv .
!cp gdrive/MyDrive/dontpatronizeme_categories.tsv .
!cp gdrive/MyDrive/train_semeval_parids-labels.csv .
!cp gdrive/MyDrive/dev_semeval_parids-labels.csv .
!cp gdrive/MyDrive/DPM_prepare.py .

In [3]:
import random
import os
from urllib import request

In [4]:
from DPM_prepare import DPM_Prepare
dpm = DPM_Prepare('.', 'dontpatronizeme_pcl.tsv')

In [5]:
dpm.load_task1()
dpm.train_task1_df.head()

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"we 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"in libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""white house press secretary sean spicer said ...",0,0
3,4,@@7811231,disabled,nz,council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" just like we received migrants fleeing el ...",0,0


In [6]:
dpm.load_task2()
dpm.train_task2_df.head()

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


,par_id,art_id,text,keyword,country,label
0,4046,@@14767805,we also know that they can benefit by receivin...,hopeless,us,"[1, 0, 0, 1, 0, 0, 0]"
1,1279,@@7896098,pope francis washed and kissed the feet of mus...,refugee,ng,"[0, 1, 0, 0, 0, 0, 0]"
2,8330,@@17252299,many refugees do n't want to be resettled anyw...,refugee,ng,"[0, 0, 1, 0, 0, 0, 0]"
3,4063,@@3002894,"""budding chefs , like """" fred """" , """" winston ...",in-need,ie,"[1, 0, 0, 1, 1, 1, 0]"
4,4089,@@25597822,"""in a 90-degree view of his constituency , one...",homeless,pk,"[1, 0, 0, 0, 0, 0, 0]"


In [7]:
dpm.load_all_positive()
dpm.all_positive_df.head(20)

,par_id,art_id,keyword,country,text,label,text_span
0,4046,@@14767805,hopeless,us,nseling from someone who can help them underst...,Authority_voice,put their situation in perspective
1,4046,@@14767805,hopeless,us,understand that their feelings are normal and ...,Unbalanced_power_relations,help them communicate with others
2,4046,@@14767805,hopeless,us,normal and that their situation is not hopele...,Unbalanced_power_relations,who could provide support
3,4046,@@14767805,hopeless,us,e and help them communicate with others who co...,Authority_voice,plan for their needs and the needs of their child
4,4046,@@14767805,hopeless,us,we also know that they can benefit by receivin...,Unbalanced_power_relations,we also know that they can benefit
5,4046,@@14767805,hopeless,us,tion is not hopeless ; someone who can help th...,Unbalanced_power_relations,someone knowledgeable about resources they can...
6,1279,@@7896098,refugee,ng,pope francis washed and kissed the feet of mus...,Shallow_solution,"washed and kissed the feet of muslim , orthodo..."
7,8330,@@17252299,refugee,ng,many refugees do n't want to be resettled anyw...,Presupposition,many refugees do n't want to be resettled anyw...
8,4063,@@3002894,in-need,ie,"""budding chefs , like """" fred """" , """" winston ...",Compassion,desperately in need of a helping hand
9,4063,@@3002894,in-need,ie,"udding chefs , like """" fred """" , """" winston """"...",Metaphors,a helping hand


In [8]:
df1 = dpm.all_positive_df
df1 = df1.rename(columns={'text': 'sentence'})
df1 = df1.assign(label=1)
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df1.shape[0]))
df1 = df1.drop(['art_id', 'keyword', 'country', 'text_span'], axis = 1)

# Display 10 random rows from the data.
df1.sample(10)

Number of training sentences: 2,760



,par_id,sentence,label
2718,7217,""""""" that 's huge , that 's humbling , """" white...",1
34,773,sheepherding in america has always been an imm...,1
1574,5806,"speaking to africa review , he also pointed ou...",1
233,8385,"""shelter chief executive polly neate said , """"...",1
1258,8053,not only will a lot of poor families have less...,1
439,3382,"""this little girl with her whole life in front...",1
148,3661,"altogether , """" he said , making it clear tha...",1
1965,9673,"neral manager , north asia , onyx hospitality ...",1
2688,7375,"as the firms ' names , locations and charges s...",1
888,10311,rement . some 1.2 million children are homeles...,1


In [9]:
dpm.all_positive_df.shape

(2760, 7)

In [10]:
dpm.load_all_negative()
dpm.all_negative_df.head()

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"we 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"in libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""white house press secretary sean spicer said ...",0,0
3,4,@@7811231,disabled,nz,council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" just like we received migrants fleeing el ...",0,0


In [11]:
dpm.all_negative_df.shape

(9476, 7)

In [12]:
df2 = dpm.all_negative_df
df2 = df2.rename(columns={'text': 'sentence'})
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df1.shape[0]))
df2 = df2.drop(['art_id', 'keyword', 'country', 'orig_label'], axis = 1)

# Display 10 random rows from the data.
df2.sample(10)

Number of training sentences: 2,760



,par_id,sentence,label
4559,5026,"if immigrants do n't pay bribes , they do n't ...",0
8840,9770,hc takes note of the hindu report on migrants,0
2249,2501,the relatively open australian economy is heav...,0
6228,6889,"very good , friendly artilce . very nice analy...",0
745,831,activists from the anna adhikar andolan have h...,0
4046,4476,"such sentiments echo that of senator hanson , ...",0
8784,9706,"""in an interview to roshmila bhattacharya for ...",0
2946,3273,lawsuit seeks release of immigrant child held ...,0
3072,3413,"1.1-million to the disabled ,",0
6211,6871,lord alton puts some of the current government...,0


In [13]:
import pandas as pd
frames = [df1, df2]
newDf = pd.concat(frames)

from sklearn.utils import shuffle
newDf = shuffle(newDf)
#newDf.sample(10)
newDf.head()
newDf.shape

(12236, 3)

In [14]:
import torch
save_file = 'preparedDataSet3.pkl'
newDf.to_pickle(save_file)
#path_google_drive = F"/content/gdrive/My Drive/{save_file}" 
#torch.save(save_file, path_google_drive)

In [15]:
from sklearn.model_selection import train_test_split

trainDF, testDF = train_test_split(newDf, test_size=0.2)

In [16]:
save_file = 'trainDataSet3.pkl'
trainDF.to_pickle(save_file)
#path_google_drive = F"/content/gdrive/My Drive/{save_file}" 
#torch.save(save_file, path_google_drive)

In [17]:
save_file = 'testDataSet3.pkl'
testDF.to_pickle(save_file)
#path_google_drive = F"/content/gdrive/My Drive/{save_file}" 
#torch.save(save_file, path_google_drive)

In [18]:
trainDFRead = pd.read_pickle("trainDataSet3.pkl")
trainDFRead.head()

,par_id,sentence,label
3012,3343,"""indeed , a number of studies have shown that ...",0
652,606,"born a christian , my humility and respect for...",1
8160,9020,"""at a recent pre-sehri coffee get-together -- ...",0
116,9518,bond went out of his way to help the less fort...,1
93,1208,alma mater- absu ... the infrastructural decay...,1


In [19]:

trainDFRead.shape

(9788, 3)

In [20]:
testDFRead = pd.read_pickle("testDataSet3.pkl")
testDFRead.head()

,par_id,sentence,label
1304,6137,"re darkness of fear , doubts , hopelessness , ...",1
6655,7373,peter mcverry trust intends to use number numb...,0
8441,9326,"the majority of these people now reside , in r...",0
4475,4935,its eric molobi housing innovation hub in sosh...,0
717,797,"""trump drew international condemnation after r...",0


In [21]:
testDFRead.shape

(2448, 3)